In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/retail-store-inventory-and-demand-forecasting/sales_data.csv


## **Data Import & Cleaning Hypotheses**

### **1. Import-Time Auto-Corrections**  
*(Automatically handled during data loading)*  

| **Issue Type**               | **Correction Applied**          | **Preserved Rows** | **Impact Analysis**                   |
|-------------------------------|----------------------------------|--------------------|---------------------------------------|
| Negative Inventory            | Clip to 0                       | Yes                | May mask true stockouts               |
| Sales > Inventory             | Cap at Inventory Level          | Yes                | Preserves volume but loses overflow   |
| Invalid Dates                 | Drop rows                       | No                 | Creates time-series gaps              |
| Numeric Overflow              | Downcast to unsigned            | Yes                | Reduces memory usage                  |

**Hypothesis**:  
> "Automated corrections during import will preserve 98%+ of rows while creating traceable artifacts for analysis."

---

### **2. Post-Import Quality Checks**  
*(Require manual validation)*  

| **Check Type**          | **Detection Method**             | **Potential Actions**                  |
|-------------------------|-----------------------------------|----------------------------------------|
| Label Consistency       | Fuzzy matching on text fields    | Standardize categories/regions         |
| Temporal Gaps           | Missing date detection           | Interpolate or flag as special events  |
| Price Inflation         | 3σ deviation from product history| Verify true inflation vs data errors   |
| Promotion Efficacy      | Lift analysis (promo vs non-promo)| Filter phantom demand from stockouts   |

**Hypothesis**:  
> "5-15% of rows will require post-import adjustments, mainly in categorical labels and promotional periods."

---

### **3. Feature Engineering Implications**

| **Observed Pattern**       | **Feature Design**               | **Rationale**                         |
|----------------------------|-----------------------------------|---------------------------------------|
| Frequent zero-demand       | Demand presence indicator        | Supports intermittent-demand models   |
| Epidemic spikes            | Shock absorption features        | Isolates exceptional events           |
| Regional price variance    | Region-price clusters            | Captures local market dynamics        |

**Hypothesis**:  
> "Auto-corrected inventory/sales relationships will require demand reconstruction features."



In [2]:
# Imports and Configuration
from pathlib import Path
import sys

DATA_PATH = Path('/kaggle/input/retail-store-inventory-and-demand-forecasting/sales_data.csv')
CHUNKSIZE = 50_000
DATE_COL = 'Date'

# Optimized dtypes
DTYPES = {
    'Store ID': 'category',
    'Product ID': 'category',
    'Category': 'category',
    'Region': 'category',
    'Inventory Level': 'uint16',
    'Units Sold': 'uint16',
    'Units Ordered': 'uint16',
    'Price': 'float32',
    'Discount': 'float32',
    'Weather Condition': 'category',
    'Promotion': 'bool',
    'Competitor Pricing': 'float32',
    'Seasonality': 'category',
    'Epidemic': 'bool',
    'Demand': 'uint16'
}

In [3]:
# Cell 2 Import Process with Detailed Tracking

def import_retail_data():
    chunks = []
    error_log = []
    stats = {
        'total_rows': 0,
        'rows_dropped': {
            'date_parsing': 0,
            'inventory_adjustment': 0,
            'sales_adjustment': 0,
            'dtype_conversion': 0,
            'chunk_failures': 0
        },
        # NEW: Track auto-corrected values
        'rows_modified': {
            'negative_inventory': 0,
            'sales_exceeding_inventory': 0,
            'invalid_dates': 0
        }
    }
    
    try:
        with pd.read_csv(
            DATA_PATH,
            chunksize=CHUNKSIZE,
            dtype=DTYPES,
            parse_dates=[DATE_COL],
            on_bad_lines='warn',
            encoding='utf-8'
        ) as reader:
            
            for chunk_idx, chunk in enumerate(reader):
                stats['total_rows'] += len(chunk)
                try:
                    # Validate required columns
                    required_cols = set(DTYPES.keys())
                    missing_cols = required_cols - set(chunk.columns)
                    if missing_cols:
                        raise ValueError(f"Missing columns: {missing_cols}")
                    
                    # Track original state
                    original_negative_inv = (chunk['Inventory Level'] < 0).sum()
                    original_over_sales = (chunk['Units Sold'] > chunk['Inventory Level']).sum()
                    
                    # Clean data - Date parsing
                    chunk[DATE_COL] = pd.to_datetime(chunk[DATE_COL], errors='coerce')
                    date_dropped = chunk[DATE_COL].isna().sum()
                    chunk = chunk.dropna(subset=[DATE_COL])
                    stats['rows_dropped']['date_parsing'] += date_dropped
                    stats['rows_modified']['invalid_dates'] += date_dropped  # NEW
                    
                    # Clean data - Inventory adjustment
                    chunk['Inventory Level'] = chunk['Inventory Level'].clip(lower=0)
                    stats['rows_modified']['negative_inventory'] += original_negative_inv  # NEW
                    stats['rows_dropped']['inventory_adjustment'] += original_negative_inv
                    
                    # Clean data - Sales adjustment
                    chunk['Units Sold'] = np.where(
                        chunk['Units Sold'] > chunk['Inventory Level'],
                        chunk['Inventory Level'],
                        chunk['Units Sold']
                    )
                    stats['rows_modified']['sales_exceeding_inventory'] += original_over_sales  # NEW
                    stats['rows_dropped']['sales_adjustment'] += original_over_sales
                    
                    # Handle numeric overflows (unchanged)
                    for col in ['Inventory Level', 'Units Sold', 'Units Ordered']:
                        try:
                            chunk[col] = pd.to_numeric(chunk[col], downcast='unsigned')
                        except:
                            pass
                    
                    chunks.append(chunk)
                    
                except Exception as e:
                    stats['rows_dropped']['chunk_failures'] += len(chunk)
                    error_log.append(f"Chunk {chunk_idx} failed: {str(e)}")
                    continue
        
        if not chunks:
            raise ValueError("No valid data chunks were processed")
            
        # Combine and finalize (unchanged)
        df = pd.concat(chunks, ignore_index=False)
        
        # Final dtype enforcement (unchanged)
        for col, dtype in DTYPES.items():
            if col in df.columns:
                try:
                    df[col] = df[col].astype(dtype)
                except Exception as e:
                    error_log.append(f"Dtype conversion failed for {col}: {str(e)}")
                    mask = pd.to_numeric(df[col], errors='coerce').isna()
                    stats['rows_dropped']['dtype_conversion'] += mask.sum()
                    df[col] = pd.to_numeric(df[col], errors='coerce')
        
        df = df.set_index(DATE_COL).sort_index()
        stats['final_rows'] = len(df)
        
        return df, error_log, stats
        
    except Exception as e:
        error_log.append(f"Fatal import error: {str(e)}")
        return None, error_log, stats


# Execute and Display Enhanced Results (Updated Output)
df, import_errors, import_stats = import_retail_data()

if df is not None:
    print(f"Successfully imported {import_stats['final_rows']:,} rows (out of {import_stats['total_rows']:,} total)")
    
    print("\nRows dropped during processing:")
    for reason, count in import_stats['rows_dropped'].items():
        print(f"- {reason.replace('_', ' ').title()}: {count:,} rows")
    
    # NEW: Display auto-corrected counts
    print("\nRows auto-corrected (not dropped):")
    for issue, count in import_stats['rows_modified'].items():
        print(f"- {issue.replace('_', ' ').title()}: {count:,} rows")
    
    print("\nSample data:")
    display(df.head())
else:
    print("Import failed")

if import_errors:
    print("\nEncountered warnings/errors:")
    for i, error in enumerate(import_errors, 1):
        print(f"{i}. {error}")

Successfully imported 76,000 rows (out of 76,000 total)

Rows dropped during processing:
- Date Parsing: 0 rows
- Inventory Adjustment: 0 rows
- Sales Adjustment: 0 rows
- Dtype Conversion: 0 rows
- Chunk Failures: 0 rows

Rows auto-corrected (not dropped):
- Negative Inventory: 0 rows
- Sales Exceeding Inventory: 0 rows
- Invalid Dates: 0 rows

Sample data:


,Store ID,Product ID,Category,Region,Inventory Level,Units Sold,Units Ordered,Price,Discount,Weather Condition,Promotion,Competitor Pricing,Seasonality,Epidemic,Demand
Date,,,,,,,,,,,,,,,
2022-01-01,S001,P0001,Electronics,North,195,102,252,72.720001,5.0,Snowy,False,85.730003,Winter,False,115
2022-01-01,S001,P0002,Clothing,North,117,117,249,80.160004,15.0,Snowy,True,92.019997,Winter,False,229
2022-01-01,S001,P0003,Clothing,North,247,114,612,62.939999,10.0,Snowy,True,60.080002,Winter,False,157
2022-01-01,S001,P0004,Electronics,North,139,45,102,87.629997,10.0,Snowy,False,85.190002,Winter,False,52
2022-01-01,S001,P0005,Groceries,North,152,65,271,54.410000,0.0,Snowy,False,51.630001,Winter,False,59


## Retail Data Quality Assessment Framework**

### **1. Objectives**
This analysis establishes a quality baseline for:
- **Forecast Reliability**: Identify data integrity issues impacting predictions
- **Pattern Discovery**: Uncover demand, sales, and inventory relationships
- **Model Guidance**: Determine appropriate modeling approaches
- **Metadata Documentation**: Create reproducible analysis standards

### **2. Assessment Dimensions**

#### **A. Core Data Validation**
**Purpose**: Ensure fundamental data integrity  
**Key Checks**:
1. **Label Consistency**  
   - Category naming conventions (e.g., "Electronics" standardization)
   - Weather condition labels (e.g., validate "Rain" vs "Raining")

2. **Temporal Integrity**  
   - Complete date sequences without gaps
   - Logical timestamp ranges (no future dates or ancient records)

3. **Value Validation**  
   - Demand/sales within plausible ranges
   - Price and discount logical relationships

#### **B. Dataset Composition**
**Purpose**: Understand data structure suitability  
**Key Metrics**:
| Dimension        | Analysis Purpose                          | Validation Approach              |
|------------------|------------------------------------------|----------------------------------|
| Products         | Forecast granularity determination       | SKU-level vs category analysis   |
| Time Coverage    | Seasonality assessment                   | 2+ years for robust patterns     |
| Locations        | Geographic representation evaluation     | Store-region alignment check     |

#### **C. Demand Analysis**
**Purpose**: Identify modeling requirements  
**Critical Examinations**:
- **Value Distribution**: Detection of artificial censoring
- **Zero-Inflation**: Intermittent demand pattern analysis
- **Event Impacts**: Epidemic and promotion effects

### **3. Actionable Insights Matrix**

| Data Quality Finding      | Analytical Impact                  | Recommended Resolution           |
|--------------------------|-----------------------------------|----------------------------------|
| Frequent zero-demand     | Intermittent demand pattern       | Croston's/TSB models            |
| Regional sales variance  | Geographic bias                   | Regional fixed effects          |
| Epidemic demand shocks   | Non-stationary time series        | Epidemic dummy variables       |
| Weather data missingness | Incomplete contextual factors     | Regional weather imputation     |

### **4. Implementation Deliverables**
1. **Data Quality Certification**
   - Priority cleaning roadmap
   - Data suitability grading

2. **Feature Development Plan**
   - Required derived features
   - Contextual data enhancements

3. **Modeling Guidelines**
   - Algorithm selection criteria
   - Special case handling procedures

In [4]:
# ================================================================
# Retail Data Quality Audit – Part A: Core Data Validation
#
# This cell performs a comprehensive audit of the raw retail data,
# focusing on:
#   1. Category label anomalies (short, rare, or catch-all terms)
#   2. Date integrity (range, gaps, duplicates, pre-epidemic records)
#   3. Demand outliers (based on configurable percentiles)
#   4. Missing data (counts, percentages, and row-level summary)
#
# Results are summarized in a quality_report dictionary and displayed
# in a format suitable for both Jupyter and script environments.
# ================================================================

# === CONFIGURABLE PARAMETERS ===
RARE_CATEGORY_THRESHOLD = 5
SHORT_LABEL_LENGTH = 3
OUTLIER_PERCENTILES = [0.01, 0.99]
EPIDEMIC_START_DATE = '2020-01-01'

# === DATA QUALITY CHECKS (PART A) ===

quality_report = {}

try:
    # 1. Category Label Analysis
    cat_series = df['Category'].astype(str).str.strip()
    category_counts = cat_series.value_counts()
    suspect_categories = category_counts[
        (category_counts.index.str.len() <= SHORT_LABEL_LENGTH) |
        (category_counts < RARE_CATEGORY_THRESHOLD) |
        (category_counts.index.str.contains('Misc|Other', case=False, regex=True))
    ]
    quality_report['category_issues'] = suspect_categories

    # 2. Date Validation
    if not isinstance(df.index, pd.DatetimeIndex):
        df.index = pd.to_datetime(df.index, errors='coerce')
    duplicate_dates = df.index.duplicated().sum()
    inferred_freq = pd.infer_freq(df.index)
    date_range = (df.index.min(), df.index.max())
    missing_dates = pd.date_range(start=date_range[0], end=date_range[1]).difference(df.index)
    pre_epidemic_records = df[df.index < EPIDEMIC_START_DATE]
    date_anomalies = {
        'date_range': date_range,
        'missing_dates': missing_dates,
        'pre_epidemic_records': pre_epidemic_records,
        'duplicate_dates': duplicate_dates,
        'inferred_frequency': inferred_freq
    }
    quality_report['date_issues'] = date_anomalies

    # 3. Demand Outlier Detection
    demand_stats = df['Demand'].describe(percentiles=OUTLIER_PERCENTILES)
    low_thresh = demand_stats[f'{int(OUTLIER_PERCENTILES[0]*100)}%']
    high_thresh = demand_stats[f'{int(OUTLIER_PERCENTILES[1]*100)}%']
    demand_outliers = df[
        (df['Demand'] < low_thresh) | 
        (df['Demand'] > high_thresh)
    ]
    quality_report['demand_outliers'] = {
        'stats': demand_stats,
        'outlier_count': len(demand_outliers),
        'min': df['Demand'].min(),
        'max': df['Demand'].max()
    }

    # 4. Missing Data Analysis
    missing_data = df.isna().sum()
    missing_percent = (missing_data / len(df)) * 100
    missing_summary = pd.DataFrame({
        'Missing Count': missing_data,
        'Missing %': missing_percent
    })
    missing_summary = missing_summary[missing_summary['Missing Count'] > 0]
    rows_with_any_missing = df.isnull().any(axis=1).sum()
    quality_report['missing_data'] = missing_summary
    quality_report['rows_with_any_missing'] = rows_with_any_missing

except Exception as e:
    print(f"Error during data quality audit: {e}")

# === DISPLAY RESULTS ===

def safe_display(obj):
    try:
        from IPython.display import display
        display(obj)
    except ImportError:
        print(obj)

print("=== DATA QUALITY REPORT ===")

print(f"\n1. Suspect Categories ({len(quality_report.get('category_issues', []))} found):")
safe_display(quality_report.get('category_issues', 'No issues found.'))

print(f"\n2. Date Issues:")
date_issues = quality_report.get('date_issues', {})
print(f"- Range: {date_issues.get('date_range', ('N/A','N/A'))[0]} to {date_issues.get('date_range', ('N/A','N/A'))[1]}")
print(f"- Missing dates: {len(date_issues.get('missing_dates', []) )}")
print(f"- Pre-epidemic records: {len(date_issues.get('pre_epidemic_records', []) )}")
print(f"- Duplicate dates: {date_issues.get('duplicate_dates', 0)}")
print(f"- Inferred frequency: {date_issues.get('inferred_frequency', 'N/A')}")

print(f"\n3. Demand Outliers (>{high_thresh:.1f} or <{low_thresh:.1f}):")
demand_out = quality_report.get('demand_outliers', {})
print(f"- {demand_out.get('outlier_count', 0)} extreme values")
print(f"- Demand min: {demand_out.get('min', 'N/A')}, max: {demand_out.get('max', 'N/A')}")

print("\n4. Missing Values by Column:")
safe_display(quality_report.get('missing_data', 'No missing values.'))
print(f"- Rows with any missing value: {quality_report.get('rows_with_any_missing', 0)}")


=== DATA QUALITY REPORT ===

1. Suspect Categories (0 found):


Series([], Name: count, dtype: int64)


2. Date Issues:
- Range: 2022-01-01 00:00:00 to 2024-01-30 00:00:00
- Missing dates: 0
- Pre-epidemic records: 0
- Duplicate dates: 75240
- Inferred frequency: None

3. Demand Outliers (>234.0 or <17.0):
- 1484 extreme values
- Demand min: 4, max: 430

4. Missing Values by Column:


,Missing Count,Missing %


- Rows with any missing value: 0


In [5]:
# ================================================================
# Retail Data Quality Audit – Part B: Dataset Composition & Structure
# ================================================================

import warnings
import logging

# Setup logging for debugging
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')

composition_report = {}

try:
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", FutureWarning)
        warnings.simplefilter("ignore", DeprecationWarning)

        # 1. Product, Category, and Store Coverage
        composition_report['n_unique_skus'] = df['Product ID'].nunique()
        composition_report['n_unique_categories'] = df['Category'].nunique()
        composition_report['n_unique_stores'] = df['Store ID'].nunique()

        # SKUs with short history (<30 records)
        sku_counts = df.groupby('Product ID', observed=True).size()
        composition_report['sku_with_short_history'] = sku_counts[sku_counts < 30]
        logging.info(f"SKUs with short history: {composition_report['sku_with_short_history'].to_dict()}")

        # SKUs with zero total demand
        sku_zero_demand = df.groupby('Product ID', observed=True)['Demand'].sum()
        composition_report['sku_zero_demand'] = sku_zero_demand[sku_zero_demand == 0]
        logging.info(f"SKUs with zero demand: {composition_report['sku_zero_demand'].to_dict()}")

        # 2. Time Coverage per SKU
        def get_time_coverage(x):
            return pd.Series({
                'start': x.index.min(),
                'end': x.index.max(),
                'n_dates': x.index.nunique()
            })
        sku_time_coverage = df.groupby('Product ID', observed=True).apply(get_time_coverage)
        composition_report['sku_time_coverage'] = sku_time_coverage

        # 3. Store/Region Representation
        store_product_coverage = df.groupby('Store ID', observed=True)['Product ID'].nunique()
        composition_report['store_product_coverage'] = store_product_coverage

        # 4. Reporting Frequency per SKU
        def infer_group_freq(x):
            try:
                freq = pd.infer_freq(x.index)
                return freq if freq is not None else 'Irregular'
            except Exception as e:
                logging.warning(f"Frequency inference failed for a group: {e}")
                return 'Error'
        sku_reporting_freq = df.groupby('Product ID', observed=True).apply(infer_group_freq).value_counts()
        composition_report['sku_reporting_freq'] = sku_reporting_freq

except Exception as e:
    logging.error(f"Error in dataset composition audit: {e}")

# === FORMATTED DISPLAY RESULTS ===
print("=== DATASET COMPOSITION REPORT ===")
print(f"{'Unique SKUs:':35}{composition_report.get('n_unique_skus', 'N/A')}")
print(f"{'Unique Categories:':35}{composition_report.get('n_unique_categories', 'N/A')}")
print(f"{'Unique Stores:':35}{composition_report.get('n_unique_stores', 'N/A')}")
print(f"{'SKUs with short history (<30):':35}{len(composition_report.get('sku_with_short_history', []))}")
print(f"{'SKUs with zero total demand:':35}{len(composition_report.get('sku_zero_demand', []))}")

print("\nStore Product Coverage (first 5):")
store_cov = composition_report.get('store_product_coverage', 'N/A')
if isinstance(store_cov, str):
    print(store_cov)
else:
    for idx, val in store_cov.head().items():
        print(f"  Store {idx}: {val} SKUs")

print("\nSKU Reporting Frequency (first 5):")
sku_freq = composition_report.get('sku_reporting_freq', 'N/A')
if isinstance(sku_freq, str):
    print(sku_freq)
else:
    for freq, count in sku_freq.head().items():
        print(f"  {freq:<10}: {count} SKUs")

print("\nSKU Time Coverage (first 5):")
sku_time_cov = composition_report.get('sku_time_coverage', 'N/A')
if isinstance(sku_time_cov, str):
    print(sku_time_cov)
else:
    for idx, row in sku_time_cov.head().iterrows():
        print(f"  SKU {idx}: {row['start'].date()} to {row['end'].date()} ({row['n_dates']} dates)")


=== DATASET COMPOSITION REPORT ===
Unique SKUs:                       20
Unique Categories:                 5
Unique Stores:                     5
SKUs with short history (<30):     0
SKUs with zero total demand:       0

Store Product Coverage (first 5):
  Store S001: 20 SKUs
  Store S002: 20 SKUs
  Store S003: 20 SKUs
  Store S004: 20 SKUs
  Store S005: 20 SKUs

SKU Reporting Frequency (first 5):
  Irregular : 20 SKUs

SKU Time Coverage (first 5):
  SKU P0001: 2022-01-01 to 2024-01-30 (760 dates)
  SKU P0002: 2022-01-01 to 2024-01-30 (760 dates)
  SKU P0003: 2022-01-01 to 2024-01-30 (760 dates)
  SKU P0004: 2022-01-01 to 2024-01-30 (760 dates)
  SKU P0005: 2022-01-01 to 2024-01-30 (760 dates)
